In [1]:
# End to End deep learning Project using simple RNN

In [88]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN,Dense, Input

In [63]:
# Load the dataset
max_features = 10000
(X_train,y_train),(X_test,y_test) = imdb.load_data(num_words=max_features)

X_train = pd.Series((v for v in [i for i in X_train]))
y_train = pd.Series((v for v in [i for i in y_train]))

X_test = pd.Series((v for v in [i for i in X_test]))
y_test = pd.Series((v for v in [i for i in y_test]))

In [64]:
#print the shape
print(f'Training Data shape:{X_train.shape} ||| Training label shape {y_train.shape}')
print(f'Testing Data shape:{X_test.shape} ||| Testing label shape {y_test.shape}')

Training Data shape:(25000,) ||| Training label shape (25000,)
Testing Data shape:(25000,) ||| Testing label shape (25000,)


In [65]:
#inspect sample review and its label
X_train

0        [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...
1        [1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...
2        [1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...
3        [1, 4, 2, 2, 33, 2804, 4, 2040, 432, 111, 153,...
4        [1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...
                               ...                        
24995    [1, 14, 9, 6, 2758, 20, 21, 1517, 7, 2078, 5, ...
24996    [1, 4679, 2784, 299, 6, 1042, 37, 80, 81, 233,...
24997    [1, 11, 6, 230, 245, 6401, 9, 6, 1225, 446, 2,...
24998    [1, 1446, 7079, 69, 72, 3305, 13, 610, 930, 8,...
24999    [1, 17, 6, 194, 337, 7, 4, 204, 22, 45, 254, 8...
Length: 25000, dtype: object

In [66]:
y_train

0        1
1        0
2        0
3        1
4        0
        ..
24995    1
24996    0
24997    0
24998    1
24999    0
Length: 25000, dtype: int64

In [67]:
X_test

0        [1, 591, 202, 14, 31, 6, 717, 10, 10, 2, 2, 5,...
1        [1, 14, 22, 3443, 6, 176, 7, 5063, 88, 12, 267...
2        [1, 111, 748, 4368, 1133, 2, 2, 4, 87, 1551, 1...
3        [1, 13, 1228, 119, 14, 552, 7, 20, 190, 14, 58...
4        [1, 40, 49, 85, 84, 1040, 146, 6, 783, 254, 43...
                               ...                        
24995    [1, 4, 274, 9, 128, 74, 4, 22, 669, 88, 7, 4, ...
24996    [1, 4, 9212, 2293, 8, 126, 67, 6, 4210, 1494, ...
24997    [1, 13, 1408, 15, 8, 135, 14, 9, 35, 32, 46, 3...
24998    [1, 11, 119, 241, 9, 4, 840, 20, 12, 468, 15, ...
24999    [1, 6, 52, 7465, 430, 22, 9, 220, 2594, 8, 28,...
Length: 25000, dtype: object

In [68]:
y_test

0        0
1        1
2        1
3        0
4        1
        ..
24995    1
24996    1
24997    0
24998    0
24999    0
Length: 25000, dtype: int64

In [70]:
sample_review = X_train[0]
sample_label = y_train[0]

print(f"Sample Review (as integers): {sample_review}")
print(f"Sample label:{sample_label}")

Sample Review (as integers): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Sample label:1


In [71]:
len(sample_review)

218

In [76]:
# Map of words index back to words for understanding
word_index = imdb.get_word_index()
word_index = {value:key for key,value in word_index.items()}

In [80]:
sample_sentence = ""
for i in sample_review:
    sample_sentence += word_index[i] + " "

In [81]:
print(sample_sentence)

the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit then have tw

In [82]:
# the above don't make any sense using tensorflow docs to decode
decoded_sample = ' '.join([word_index.get(i-3,'?') for i in sample_review])

In [ ]:
print(decoded_sample)
# this looks meaningful

? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

In [86]:
max_len = 500
X_train = sequence.pad_sequences(X_train, maxlen = max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

In [87]:
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]])

In [99]:
# Training Simple RNN
model = Sequential()
model.add(Input(shape=(max_len,)))
model.add(Embedding(max_features,128))
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [100]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [101]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [105]:
from tensorflow.keras.callbacks import EarlyStopping
earlystop  = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
earlystop

In [106]:
history = model.fit(X_train,y_train,epochs=10,batch_size=32,validation_split=0.2,callbacks=[earlystop])

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 57s 92ms/step - accuracy: 0.6487 - loss: 0.6387 - val_accuracy: 0.6522 - val_loss: 0.6271
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 61s 98ms/step - accuracy: 0.6980 - loss: 0.5923 - val_accuracy: 0.6732 - val_loss: 0.5993
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 62s 99ms/step - accuracy: 0.7453 - loss: 0.5234 - val_accuracy: 0.6998 - val_loss: 0.5722
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 63s 101ms/step - accuracy: 0.7856 - loss: 0.4574 - val_accuracy: 0.7016 - val_loss: 0.5675
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 65s 103ms/step - accuracy: 0.8205 - loss: 0.4018 - val_accuracy: 0.7334 - val_loss: 0.5506
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 65s 105ms/step - accuracy: 0.8405 - loss: 0.3835 - val_accuracy: 0.7434 - val_loss: 0.5790
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 64s 102ms/step - accuracy: 0.8785 - loss: 0.2992 - val_accuracy: 0.7468 - val_loss: 0.5870
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 65s 104ms/step - accuracy: 0.9027 - loss: 0.25

In [107]:
#Save the model file
model.save("Simple_rnn_Imdb.h5")